In [1]:
import pandas as pd
import numpy as np
import os
import re
from icecream import ic
import copy
import ast
import json

In [2]:
import warnings
warnings.filterwarnings("ignore")

### check 3D MRIs

In [13]:
def add_mris(mri_path, new_mris):
    for path, subdirs, files in os.walk(mri_path):
        for name in files:
            if '.zip' not in name and '.py' not in name and '.sh' not in name and (name.endswith('.nii') or name.endswith('.json')):
                for word in path.split('/'):
                    if word.endswith('ni'):
                        mri_folder_name = word
                        break
                # mri_name = path.split('/')
                # print(path)

                mri_name = '.'.join(name.split('.')[:-1])
                # print(mri_name)

                if mri_folder_name not in new_mris:
                    new_mris[mri_folder_name] = defaultdict()

                if mri_name not in new_mris[mri_folder_name]:
                    new_mris[mri_folder_name][mri_name] = defaultdict()
                    
                full_path = path + '/' + name

                if os.path.getsize(full_path) == 0:
                    continue

                if name.endswith('nii'):
                    if ('mri' not in new_mris[mri_folder_name][mri_name]):
                        new_mris[mri_folder_name][mri_name]['mri'] = full_path
                elif name.endswith('json'):
                    if ('json' not in new_mris[mri_folder_name][mri_name]):
                        new_mris[mri_folder_name][mri_name]['json'] = full_path

    missing_mris = []
    for i, (mri_folder_name, mri_dict) in enumerate(new_mris.items()):
        for (mri_name, val) in mri_dict.items(): 
            if ('mri' not in new_mris[mri_folder_name][mri_name]) or ('json' not in new_mris[mri_folder_name][mri_name]):
                missing_mris.append((mri_folder_name, mri_name))
                
    for (mri_folder_name, mri_name) in missing_mris:         
        new_mris[mri_folder_name].pop(mri_name)
         
    return new_mris

In [14]:
from collections import defaultdict

mri_path = '/SeaExpCIFS/NACC_updated/raw/'
all_new_mris = defaultdict()

all_new_mris = add_mris(mri_path + 'T1', all_new_mris)
all_new_mris = add_mris(mri_path + 'T2', all_new_mris)
all_new_mris = add_mris(mri_path + 'DTI', all_new_mris)
all_new_mris = add_mris(mri_path + 'DWI', all_new_mris)
all_new_mris = add_mris(mri_path + 'FLAIR', all_new_mris)


In [15]:
import json
with open('/home/skowshik/ADRD_repo/pipeline_v1_temp/clinician_review/mri_paths.json', 'w') as json_file:
    json.dump(all_new_mris, json_file, indent=4)

In [23]:
file_path = '/home/skowshik/ADRD_repo/pipeline_v1_temp/clinician_review/mri_paths.json'
with open(file_path, 'r') as json_file:
    all_new_mris = json.load(json_file)

In [24]:
from tqdm import tqdm
mri_3d = {'mag_3t': {'t1': {}, 't2': {}, 'dti': {}, 'dwi': {}, 'swi': {}, 'flair': {}, 'unknown': {}}, 'mag_15t': {'t1': {}, 't2': {}, 'dti': {}, 'dwi': {}, 'swi': {}, 'flair': {}, 'unknown': {}}, 'mag_unk': {'t1': {}, 't2': {}, 'dti': {}, 'dwi': {}, 'swi': {}, 'flair': {}, 'unknown': {}}}
mri_2d = {'mag_3t': {'t1': {}, 't2': {}, 'dti': {}, 'dwi': {}, 'swi': {}, 'flair': {}, 'unknown': {}}, 'mag_15t': {'t1': {}, 't2': {}, 'dti': {}, 'dwi': {}, 'swi': {}, 'flair': {}, 'unknown': {}}, 'mag_unk': {'t1': {}, 't2': {}, 'dti': {}, 'dwi': {}, 'swi': {}, 'flair': {}, 'unknown': {}}}
mri_unk = {'mag_3t': {'t1': {}, 't2': {}, 'dti': {}, 'dwi': {}, 'swi': {}, 'flair': {}, 'unknown': {}}, 'mag_15t': {'t1': {}, 't2': {}, 'dti': {}, 'dwi': {}, 'swi': {}, 'flair': {}, 'unknown': {}}, 'mag_unk': {'t1': {}, 't2': {}, 'dti': {}, 'dwi': {}, 'swi': {}, 'flair': {}, 'unknown': {}}}

for i, (mri, mri_dict) in tqdm(enumerate(all_new_mris.items())):
    for (name, val) in mri_dict.items():
        # try:
        with open(val['json'], 'r') as fi:
            data = json.load(fi)
            if 'MagneticFieldStrength' in data:
                mag_field = data["MagneticFieldStrength"]
                if (mag_field == 1.5) or (mag_field == 15000):
                    mag = 'mag_15t'
                else:
                    mag = 'mag_3t'
            else:
                mag = 'mag_unk'
            src_mri = val['mri']
            src_json = val['json']
            mri_dict = {'mri': src_mri, 'json': src_json}
            if 'SeriesDescription' in data:
                series = data['SeriesDescription']
                if 'swi' in series.lower():
                    seq_type = 'swi'
                elif 'flair' in series.lower():
                    seq_type = 'flair'
                elif 'dti' in series.lower():
                    seq_type = 'dti'
                elif 'dwi' in series.lower():
                    seq_type = 'dwi'
                elif 't2' in series.lower():
                    seq_type = 't2'
                elif 't1' in series.lower() or 'mprage' in series.lower() or 'bravo' in series.lower() or 'spgr' in series.lower() or 'fspgr' in series.lower() or 'mp-rage' in series.lower():
                    seq_type = 't1'
                else:
                    seq_type = 'unknown'
            else:
                seq_type = 'unknown'

            if 'MRAcquisitionType' in data:
                if data['MRAcquisitionType'] == '3D':
                    if mri in mri_3d[mag][seq_type]:
                        mri_3d[mag][seq_type][mri].append(mri_dict)
                    else:
                        mri_3d[mag][seq_type][mri] = [mri_dict]
                elif data['MRAcquisitionType'] == '2D':
                    if mri in mri_2d[mag][seq_type]:
                        mri_2d[mag][seq_type][mri].append(mri_dict)
                    else:
                        mri_2d[mag][seq_type][mri] = [mri_dict]
                else:
                    if mri in mri_unk[mag][seq_type]:
                        mri_unk[mag][seq_type][mri].append(mri_dict)
                    else:
                        mri_unk[mag][seq_type][mri] = [mri_dict]
            else:
                if mri in mri_unk[mag][seq_type]:
                    mri_unk[mag][seq_type][mri].append(mri_dict)
                else:
                    mri_unk[mag][seq_type][mri] = [mri_dict]
        # except:
        #     continue

4600it [06:13, 12.32it/s]


In [25]:
len(mri_3d['mag_3t']['swi']) + len(mri_3d['mag_15t']['swi'])

63

In [26]:
with open('/home/skowshik/ADRD_repo/pipeline_v1_temp/clinician_review/mri_3d.json', 'w') as json_file:
    json.dump(mri_3d, json_file, indent=4)

with open('/home/skowshik/ADRD_repo/pipeline_v1_temp/clinician_review/mri_2d.json', 'w') as json_file:
    json.dump(mri_2d, json_file, indent=4)

with open('/home/skowshik/ADRD_repo/pipeline_v1_temp/clinician_review/mri_unk.json', 'w') as json_file:
    json.dump(mri_unk, json_file, indent=4)

### pick 10 cases for each label 3d MRIs

In [28]:
mri_3d_path = '/home/skowshik/ADRD_repo/pipeline_v1_temp/clinician_review/mri_3d.json'
mri_2d_path = '/home/skowshik/ADRD_repo/pipeline_v1_temp/clinician_review/mri_2d.json'
mri_unk_path = '/home/skowshik/ADRD_repo/pipeline_v1_temp/clinician_review/mri_unk.json'

with open(mri_3d_path, 'r') as json_file:
    mri_3d = json.load(json_file)

with open(mri_2d_path, 'r') as json_file:
    mri_2d = json.load(json_file)

with open(mri_unk_path, 'r') as json_file:
    mri_unk = json.load(json_file)

file_path = '/home/skowshik/ADRD_repo/pipeline_v1_temp/clinician_review/mri_paths.json'
with open(file_path, 'r') as json_file:
    all_new_mris = json.load(json_file)

In [29]:
avail_mris = []
for i, (mri, mri_dict) in enumerate(all_new_mris.items()):
    avail_mris.append(mri)
avail_mris = [mri[:-2] + '.zip' for mri in avail_mris]

In [30]:
avail_3d_mris = list(set(list(mri_3d['mag_3t']['t1'].keys()) + list(mri_3d['mag_3t']['t2'].keys()) + list(mri_3d['mag_3t']['dti'].keys()) + list(mri_3d['mag_3t']['dwi'].keys()) + list(mri_3d['mag_3t']['flair'].keys()) + list(mri_3d['mag_3t']['swi'].keys()) + list(mri_3d['mag_3t']['unknown'].keys()) + list(mri_3d['mag_15t']['t1'].keys()) + list(mri_3d['mag_15t']['t2'].keys()) + list(mri_3d['mag_15t']['dti'].keys()) + list(mri_3d['mag_15t']['dwi'].keys()) + list(mri_3d['mag_15t']['flair'].keys()) + list(mri_3d['mag_15t']['swi'].keys()) + list(mri_3d['mag_15t']['unknown'].keys()) + list(mri_3d['mag_unk']['t1'].keys()) + list(mri_3d['mag_unk']['t2'].keys()) + list(mri_3d['mag_unk']['dti'].keys()) + list(mri_3d['mag_unk']['dwi'].keys()) + list(mri_3d['mag_unk']['flair'].keys()) + list(mri_3d['mag_unk']['swi'].keys()) + list(mri_3d['mag_unk']['unknown'].keys())))
avail_3d_mris = [mri[:-2] + '.zip' for mri in avail_3d_mris]

In [31]:
nacc_mri_3d = nacc[nacc['filename'].isin(avail_3d_mris)]
new_nacc_mri_3d = new_nacc[new_nacc['filename'].isin(avail_3d_mris)]
nacc_mri_all = nacc[nacc['filename'].isin(avail_mris)]
new_nacc_mri_all = new_nacc[new_nacc['filename'].isin(avail_mris)]

In [32]:
len(new_nacc_mri_3d)

2270

In [33]:
len(new_nacc_mri_all)

2612

In [34]:
avail_mri_cnt_3d_new = {}
for label in ['NC', 'MCI', 'DE', 'AD', 'LBD', 'VD', 'PRD', 'FTD', 'NPH', 'SEF', 'PSY', 'TBI', 'ODE']:
    avail_mri_cnt_3d_new[label] = dict(new_nacc_mri_3d[label].value_counts())
avail_mri_cnt_3d_new

{'NC': {1: 1153, 0: 1117},
 'MCI': {0: 1924, 1: 346},
 'DE': {0: 1499, 1: 771},
 'AD': {0: 1641, 1: 629},
 'LBD': {0: 2194, 1: 76},
 'VD': {0: 2216, 1: 54},
 'PRD': {0: 2262, 1: 8},
 'FTD': {0: 2203, 1: 67},
 'NPH': {0: 2263, 1: 7},
 'SEF': {0: 2254, 1: 16},
 'PSY': {0: 2222, 1: 48},
 'TBI': {0: 2265, 1: 5},
 'ODE': {0: 2223, 1: 47}}

In [35]:
avail_mri_cnt_all_new = {}
for label in ['NC', 'MCI', 'DE', 'AD', 'LBD', 'VD', 'PRD', 'FTD', 'NPH', 'SEF', 'PSY', 'TBI', 'ODE']:
    avail_mri_cnt_all_new[label] = dict(new_nacc_mri_all[label].value_counts())
avail_mri_cnt_all_new

{'NC': {0: 1316, 1: 1296},
 'MCI': {0: 2213, 1: 399},
 'DE': {0: 1695, 1: 917},
 'AD': {0: 1871, 1: 741},
 'LBD': {0: 2518, 1: 94},
 'VD': {0: 2548, 1: 64},
 'PRD': {0: 2597, 1: 15},
 'FTD': {0: 2533, 1: 79},
 'NPH': {0: 2604, 1: 8},
 'SEF': {0: 2592, 1: 20},
 'PSY': {0: 2540, 1: 72},
 'TBI': {0: 2605, 1: 7},
 'ODE': {0: 2556, 1: 56}}

### Neurologist cases

In [32]:
radio_cases_udprs = pd.read_csv('/data_1/clinician_review/Radiologist/clinician_review_cases_proc_fea_radio.csv')

In [33]:
radio_cases = radio_cases_udprs.rename(columns=lambda x: x.replace('udprs', 'updrs') if x.startswith('udprs') else x)

In [35]:
radio_cases

,filename,ID,non_null_count,MRI_CNT,NC,IMCI,MCI,DE,AD,LBD,...,bat_RESPDIST,bat_RESPINTR,bat_RESPDISN,bat_RESPFATG,bat_RESPEMOT,bat_RESPASST,bat_RESPOTH,bat_RESPOTHX,apoe_NACCNE4S,case_number
0,mri6873.zip,NACC433881,198,4.0,0,0,0,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CASE_1
1,NACC670524_12840113696323972500152388820101028...,NACC670524,183,3.0,0,0,0,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASE_2
2,NACC698052_21684011141514153893144971165483.zip,NACC698052,191,2.0,0,0,0,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASE_3
3,NACC877796_12840113745101000100200039925398826...,NACC877796,201,4.0,0,0,0,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_4
4,mri3617.zip,NACC884172,170,6.0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,mri1880.zip,NACC590625,198,5.0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_66
66,mri3582.zip,NACC769021,196,6.0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_67
67,mri3573.zip,NACC318822,191,6.0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASE_68
68,NACC209327_12840113745101000100200040113592627...,NACC209327,200,4.0,0,0,0,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_69


In [37]:
len(list(radio_cases.columns))

464

In [38]:
new_nacc_mri_3d_sorted = new_nacc_mri_3d.sort_values(['MRI_CNT', 'non_null_count'], ascending=False)

In [39]:
cog_labels = ['NC', 'MCI']
de_labels = ['TBI', 'NPH', 'PRD', 'AD', 'LBD', 'VD', 'FTD', 'SEF', 'PSY', 'ODE']
random_state = 1
picked_cases = [radio_cases]
picked_case_ids = list(radio_cases['ID'])
# picked_cases = []
# picked_case_ids = []
for label in cog_labels:
    # print(label)
    df = new_nacc_mri_3d_sorted[~new_nacc_mri_3d_sorted['ID'].isin(picked_case_ids)]
    # picked = df[df[label] == 1].sample(n=10, random_state=random_state)
    picked = df[df[label] == 1][:15]
    picked_cases.append(picked)
    picked_case_ids += list(picked['ID'])
# for label in de_labels:
#     # print(label)
#     if label == 'TBI':
#         df = new_nacc_mri_all[~new_nacc_mri_all['ID'].isin(picked_case_ids)]
#     else:
#         df = new_nacc_mri_3d_sorted[~new_nacc_mri_3d_sorted['ID'].isin(picked_case_ids)]
#     # picked = df[df[label] == 1].sample(n=7, random_state=random_state)
#     picked = df[df[label] == 1][:7]
#     picked_cases.append(picked)
#     picked_case_ids += list(picked['ID'])

cli_cases_neuro = pd.concat(picked_cases, ignore_index=True)
cli_cases_neuro = cli_cases_neuro.sample(frac=1, random_state=random_state).reset_index(drop=True)
cli_cases_neuro.drop_duplicates(inplace=True)
cli_cases_neuro['case_number'] = 'CASE_' + (cli_cases_neuro.index + 1).astype(str)

In [40]:
cli_cases_neuro

,filename,ID,non_null_count,MRI_CNT,NC,IMCI,MCI,DE,AD,LBD,...,bat_RESPDIST,bat_RESPINTR,bat_RESPDISN,bat_RESPFATG,bat_RESPEMOT,bat_RESPASST,bat_RESPOTH,bat_RESPOTHX,apoe_NACCNE4S,case_number
0,mri5502.zip,NACC425502,204,5.0,1,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_1
1,mri5263.zip,NACC784242,204,5.0,1,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_2
2,mri3615.zip,NACC158963,199,6.0,0,0.0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_3
3,mri5469.zip,NACC464247,204,5.0,1,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_4
4,mri5397.zip,NACC087355,204,5.0,0,NaN,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CASE_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,mri5000.zip,NACC827850,205,5.0,1,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_96
96,NACC630130_12840113619213417625342834029133452...,NACC630130,197,3.0,0,0.0,0,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CASE_97
97,mri5405.zip,NACC475721,205,5.0,1,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_98
98,mri3259.zip,NACC311767,181,6.0,0,0.0,0,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_99


In [67]:
# cli_cases_neuro = pd.concat([cli_cases_neuro, new_nacc_mri_3d_sorted[new_nacc_mri_3d_sorted['ID'] == 'NACC043867']], ignore_index = True)
# cli_cases_neuro = cli_cases_neuro.drop(cli_cases_neuro[cli_cases_neuro['case_number'] == 'CASE_24'].index)
# cli_cases_neuro.loc[cli_cases_neuro[cli_cases_neuro['ID'] == 'NACC043867'].index, 'case_number'] = 'CASE_24'
# cli_cases_neuro

In [41]:
len(cli_cases_neuro[~cli_cases_neuro['bat_MOCATOTS'].isna()])

9

In [69]:
# cli_cases_neuro = cli_cases_neuro.drop(['IMCI'], axis=1)

In [42]:
len(set(cli_cases_neuro['ID']))

100

In [43]:
for label in ['NC', 'MCI', 'DE', 'AD', 'LBD', 'VD', 'PRD', 'FTD', 'NPH', 'SEF', 'PSY', 'TBI', 'ODE']:
    print(cli_cases_neuro[label].value_counts())

0    85
1    15
Name: NC, dtype: int64
0    85
1    15
Name: MCI, dtype: int64
1    70
0    30
Name: DE, dtype: int64
0    59
1    41
Name: AD, dtype: int64
0    91
1     9
Name: LBD, dtype: int64
0    89
1    11
Name: VD, dtype: int64
0    93
1     7
Name: PRD, dtype: int64
0    89
1    11
Name: FTD, dtype: int64
0    93
1     7
Name: NPH, dtype: int64
0    92
1     8
Name: SEF, dtype: int64
0    85
1    15
Name: PSY, dtype: int64
0    93
1     7
Name: TBI, dtype: int64
0    91
1     9
Name: ODE, dtype: int64


In [50]:
# x = pd.read_csv('/data_1/clinician_review/Neurologist/clinician_review_cases_proc_fea_neuro.csv')
# for i, row in x.iterrows():
#     if row['filename'] != cli_cases_neuro.iloc[i]['filename']:
#         print(i)

In [51]:
cli_cases_neuro.to_csv('/data_1/clinician_review/Neurologist/clinician_review_cases_proc_fea_neuro.csv', index=False)

In [52]:
columns = ['filename', 'ID', 'case_number', 'NC', 'MCI', 'AD', 'LBD', 'VD', 'PRD', 'FTD', 'NPH', 'SEF', 'PSY', 'TBI', 'ODE', 'his_NACCREAS', 'his_NACCREFR', 'his_BIRTHMO', 'his_BIRTHYR', 'his_HISPANIC', 'his_HISPOR', 'his_RACE', 'his_RACESEC', 'his_RACETER', 'his_PRIMLANG', 'his_EDUC', 'his_MARISTAT', 'his_LIVSIT', 'his_INDEPEND', 'his_RESIDENC', 'his_HANDED', 'his_NACCAGE', 'his_NACCAGEB', 'his_NACCNIHR', 'his_NACCFAM', 'his_NACCMOM', 'his_NACCDAD', 'his_NACCFADM', 'his_NACCAM', 'his_NACCAMS', 'his_NACCFFTD', 'his_NACCFM', 'his_NACCFMS', 'his_NACCOM', 'his_NACCOMS', 'his_TOBAC30', 'his_TOBAC100', 'his_SMOKYRS', 'his_PACKSPER', 'his_QUITSMOK', 'his_ALCOCCAS', 'his_ALCFREQ', 'his_CVHATT', 'his_HATTMULT', 'his_HATTYEAR', 'his_CVAFIB', 'his_CVANGIO', 'his_CVBYPASS', 'his_CVPACDEF', 'his_CVPACE', 'his_CVCHF', 'his_CVANGINA', 'his_CVHVALVE', 'his_CVOTHR', 'his_CBSTROKE', 'his_STROKMUL', 'his_NACCSTYR', 'his_CBTIA', 'his_TIAMULT', 'his_NACCTIYR', 'his_PD', 'his_PDYR', 'his_PDOTHR', 'his_PDOTHRYR', 'his_SEIZURES', 'his_TBI', 'his_TBIBRIEF', 'his_TRAUMBRF', 'his_TBIEXTEN', 'his_TRAUMEXT', 'his_TBIWOLOS', 'his_TRAUMCHR', 'his_TBIYEAR', 'his_NCOTHR', 'his_DIABETES', 'his_DIABTYPE', 'his_HYPERTEN', 'his_HYPERCHO', 'his_B12DEF', 'his_THYROID', 'his_ARTHRIT', 'his_ARTHTYPE', 'his_ARTHSPIN', 'his_ARTHUNK', 'his_INCONTU', 'his_INCONTF', 'his_APNEA', 'his_RBD', 'his_INSOMN', 'his_OTHSLEEP', 'his_ALCOHOL', 'his_ABUSOTHR', 'his_PTSD', 'his_BIPOLAR', 'his_SCHIZ', 'his_DEP2YRS', 'his_DEPOTHR', 'his_ANXIETY', 'his_OCD', 'his_NPSYDEV', 'his_PSYCDIS', 'his_NACCTBI', 'med_ANYMEDS', 'med_NACCAMD', 'med_NACCAHTN', 'med_NACCHTNC', 'med_NACCACEI', 'med_NACCAAAS', 'med_NACCBETA', 'med_NACCCCBS', 'med_NACCDIUR', 'med_NACCVASD', 'med_NACCANGI', 'med_NACCLIPL', 'med_NACCNSD', 'med_NACCAC', 'med_NACCADEP', 'med_NACCAPSY', 'med_NACCADMD', 'med_NACCPDMD', 'med_NACCEMD', 'med_NACCEPMD', 'med_NACCDBMD', 'ph_HEIGHT', 'ph_WEIGHT', 'ph_NACCBMI', 'ph_BPSYS', 'ph_BPDIAS', 'ph_HRATE', 'ph_VISION', 'ph_VISCORR', 'ph_VISWCORR', 'ph_HEARING', 'ph_HEARAID', 'ph_HEARWAID', 'cvd_ABRUPT', 'cvd_STEPWISE', 'cvd_SOMATIC', 'cvd_EMOT', 'cvd_HXHYPER', 'cvd_HXSTROKE', 'cvd_FOCLSYM', 'cvd_FOCLSIGN', 'cvd_HACHIN', 'cvd_CVDCOG', 'cvd_STROKCOG', 'cvd_CVDIMAG', 'cvd_CVDIMAG1', 'cvd_CVDIMAG2', 'cvd_CVDIMAG3', 'cvd_CVDIMAG4', 'updrs_PDNORMAL', 'updrs_SPEECH', 'updrs_FACEXP', 'updrs_TRESTFAC', 'updrs_TRESTRHD', 'updrs_TRESTLHD', 'updrs_TRESTRFT', 'updrs_TRESTLFT', 'updrs_TRACTRHD', 'updrs_TRACTLHD', 'updrs_RIGDNECK', 'updrs_RIGDUPRT', 'updrs_RIGDUPLF', 'updrs_RIGDLORT', 'updrs_RIGDLOLF', 'updrs_TAPSRT', 'updrs_TAPSLF', 'updrs_HANDMOVR', 'updrs_HANDMOVL', 'updrs_HANDALTR', 'updrs_HANDALTL', 'updrs_LEGRT', 'updrs_LEGLF', 'updrs_ARISING', 'updrs_POSTURE', 'updrs_GAIT', 'updrs_POSSTAB', 'updrs_BRADYKIN', 'npiq_NPIQINF', 'npiq_DEL', 'npiq_HALL', 'npiq_AGIT', 'npiq_DEPD', 'npiq_ELAT', 'npiq_APA', 'npiq_DISN', 'npiq_IRR', 'npiq_MOT', 'npiq_NITE', 'npiq_APP', 'gds_NOGDS', 'gds_SATIS', 'gds_DROPACT', 'gds_EMPTY', 'gds_BORED', 'gds_SPIRITS', 'gds_AFRAID', 'gds_HAPPY', 'gds_HELPLESS', 'gds_STAYHOME', 'gds_MEMPROB', 'gds_WONDRFUL', 'gds_WRTHLESS', 'gds_ENERGY', 'gds_HOPELESS', 'gds_BETTER', 'gds_NACCGDS', 'faq_BILLS', 'faq_TAXES', 'faq_SHOPPING', 'faq_GAMES', 'faq_STOVE', 'faq_MEALPREP', 'faq_EVENTS', 'faq_PAYATTN', 'faq_REMDATES', 'faq_TRAVEL', 'exam_NORMEXAM', 'exam_FOCLDEF', 'exam_GAITDIS', 'exam_EYEMOVE', 'exam_PARKSIGN', 'exam_RESTTRL', 'exam_RESTTRR', 'exam_SLOWINGL', 'exam_SLOWINGR', 'exam_RIGIDL', 'exam_RIGIDR', 'exam_BRADY', 'exam_PARKGAIT', 'exam_POSTINST', 'exam_CVDSIGNS', 'exam_CORTDEF', 'exam_SIVDFIND', 'exam_CVDMOTL', 'exam_CVDMOTR', 'exam_CORTVISL', 'exam_CORTVISR', 'exam_SOMATL', 'exam_SOMATR', 'exam_POSTCORT', 'exam_PSPCBS', 'exam_EYEPSP', 'exam_DYSPSP', 'exam_AXIALPSP', 'exam_GAITPSP', 'exam_APRAXSP', 'exam_APRAXL', 'exam_APRAXR', 'exam_CORTSENL', 'exam_CORTSENR', 'exam_ATAXL', 'exam_ATAXR', 'exam_ALIENLML', 'exam_ALIENLMR', 'exam_DYSTONL', 'exam_DYSTONR', 'exam_MYOCLLT', 'exam_MYOCLRT', 'exam_ALSFIND', 'exam_GAITNPH', 'exam_OTHNEUR', 'bat_MMSECOMP', 'bat_MMSELOC', 'bat_MMSELAN', 'bat_MMSEVIS', 'bat_MMSEHEAR', 'bat_MMSEORDA', 'bat_MMSEORLO', 'bat_PENTAGON', 'bat_NACCMMSE', 'bat_NPSYCLOC', 'bat_NPSYLAN', 'bat_LOGIMO', 'bat_LOGIDAY', 'bat_LOGIYR', 'bat_LOGIPREV', 'bat_LOGIMEM', 'bat_MEMUNITS', 'bat_MEMTIME', 'bat_UDSBENTC', 'bat_UDSBENTD', 'bat_UDSBENRS', 'bat_DIGIF', 'bat_DIGIFLEN', 'bat_DIGIB', 'bat_DIGIBLEN', 'bat_ANIMALS', 'bat_VEG', 'bat_TRAILA', 'bat_TRAILARR', 'bat_TRAILALI', 'bat_TRAILB', 'bat_TRAILBRR', 'bat_TRAILBLI', 'bat_WAIS', 'bat_BOSTON', 'bat_UDSVERFC', 'bat_UDSVERFN', 'bat_UDSVERNF', 'bat_UDSVERLC', 'bat_UDSVERLR', 'bat_UDSVERLN', 'bat_UDSVERTN', 'bat_UDSVERTE', 'bat_UDSVERTI', 'bat_COGSTAT', 'bat_MODCOMM', 'bat_MOCACOMP', 'bat_MOCAREAS', 'bat_MOCALOC', 'bat_MOCALAN', 'bat_MOCAVIS', 'bat_MOCAHEAR', 'bat_MOCATOTS', 'bat_NACCMOCA', 'bat_MOCATRAI', 'bat_MOCACUBE', 'bat_MOCACLOC', 'bat_MOCACLON', 'bat_MOCACLOH', 'bat_MOCANAMI', 'bat_MOCAREGI', 'bat_MOCADIGI', 'bat_MOCALETT', 'bat_MOCASER7', 'bat_MOCAREPE', 'bat_MOCAFLUE', 'bat_MOCAABST', 'bat_MOCARECN', 'bat_MOCARECC', 'bat_MOCARECR', 'bat_MOCAORDT', 'bat_MOCAORMO', 'bat_MOCAORYR', 'bat_MOCAORDY', 'bat_MOCAORPL', 'bat_MOCAORCT', 'bat_CRAFTVRS', 'bat_CRAFTURS', 'bat_DIGFORCT', 'bat_DIGFORSL', 'bat_DIGBACCT', 'bat_DIGBACLS', 'bat_CRAFTDVR', 'bat_CRAFTDRE', 'bat_CRAFTDTI', 'bat_CRAFTCUE', 'bat_MINTTOTS', 'bat_MINTTOTW', 'bat_MINTSCNG', 'bat_MINTSCNC', 'bat_MINTPCNG', 'bat_MINTPCNC', 'bat_MOCBTOTS', 'bat_NACCMOCB', 'bat_REY1REC', 'bat_REY1INT', 'bat_REY2REC', 'bat_REY2INT', 'bat_REY3REC', 'bat_REY3INT', 'bat_REY4REC', 'bat_REY4INT', 'bat_REY5REC', 'bat_REY5INT', 'bat_REY6REC', 'bat_REY6INT', 'bat_OTRAILA', 'bat_OTRLARR', 'bat_OTRLALI', 'bat_OTRAILB', 'bat_OTRLBRR', 'bat_OTRLBLI', 'bat_REYDREC', 'bat_REYDINT', 'bat_REYTCOR', 'bat_REYFPOS', 'bat_VNTTOTW', 'bat_VNTPCNC', 'bat_RESPVAL', 'bat_RESPHEAR', 'bat_RESPDIST', 'bat_RESPINTR', 'bat_RESPDISN', 'bat_RESPFATG', 'bat_RESPEMOT', 'bat_RESPASST', 'bat_RESPOTH', 'apoe_NACCNE4S', 'his_SEX', 'npiq_ANX', 'his_ARTHUPEX', 'his_ARTHLOEX', 'med_NACCAANX', 'exam_NACCNREX']
nacc_orig_columns = ['filename', 'ID', 'non_null_count', 'case_number', 'MRI_CNT', 'NC_lb', 'MCI_lb', 'DE_lb', 'AD_lb', 'LBD_lb', 'VD_lb', 'PRD_lb', 'FTD_lb', 'NPH_lb', 'SEF_lb', 'PSY_lb', 'TBI_lb', 'ODE_lb', 'NACCREAS', 'NACCREFR', 'BIRTHMO', 'BIRTHYR', 'HISPANIC', 'HISPOR', 'RACE', 'RACESEC', 'RACETER', 'PRIMLANG', 'EDUC', 'MARISTAT', 'NACCLIVS', 'INDEPEND', 'RESIDENC', 'HANDED', 'NACCAGE', 'NACCAGEB', 'NACCNIHR', 'NACCFAM', 'NACCMOM', 'NACCDAD', 'NACCFADM', 'NACCAM', 'NACCAMS', 'NACCFFTD', 'NACCFM', 'NACCFMS', 'NACCOM', 'NACCOMS', 'TOBAC30', 'TOBAC100', 'SMOKYRS', 'PACKSPER', 'QUITSMOK', 'ALCOCCAS', 'ALCFREQ', 'CVHATT', 'HATTMULT', 'HATTYEAR', 'CVAFIB', 'CVANGIO', 'CVBYPASS', 'CVPACDEF', 'CVPACE', 'CVCHF', 'CVANGINA', 'CVHVALVE', 'CVOTHR', 'CBSTROKE', 'STROKMUL', 'NACCSTYR', 'CBTIA', 'TIAMULT', 'NACCTIYR', 'PD', 'PDYR', 'PDOTHR', 'PDOTHRYR', 'SEIZURES', 'TBI', 'TBIBRIEF', 'TRAUMBRF', 'TBIEXTEN', 'TRAUMEXT', 'TBIWOLOS', 'TRAUMCHR', 'TBIYEAR', 'NCOTHR', 'DIABETES', 'DIABTYPE', 'HYPERTEN', 'HYPERCHO', 'B12DEF', 'THYROID', 'ARTHRIT', 'ARTHTYPE', 'ARTHSPIN', 'ARTHUNK', 'INCONTU', 'INCONTF', 'APNEA', 'RBD', 'INSOMN', 'OTHSLEEP', 'ALCOHOL', 'ABUSOTHR', 'PTSD', 'BIPOLAR', 'SCHIZ', 'DEP2YRS', 'DEPOTHR', 'ANXIETY', 'OCD', 'NPSYDEV', 'PSYCDIS', 'NACCTBI', 'ANYMEDS', 'NACCAMD', 'NACCAHTN', 'NACCHTNC', 'NACCACEI', 'NACCAAAS', 'NACCBETA', 'NACCCCBS', 'NACCDIUR', 'NACCVASD', 'NACCANGI', 'NACCLIPL', 'NACCNSD', 'NACCAC', 'NACCADEP', 'NACCAPSY', 'NACCADMD', 'NACCPDMD', 'NACCEMD', 'NACCEPMD', 'NACCDBMD', 'HEIGHT', 'WEIGHT', 'NACCBMI', 'BPSYS', 'BPDIAS', 'HRATE', 'VISION', 'VISCORR', 'VISWCORR', 'HEARING', 'HEARAID', 'HEARWAID', 'ABRUPT', 'STEPWISE', 'SOMATIC', 'EMOT', 'HXHYPER', 'HXSTROKE', 'FOCLSYM', 'FOCLSIGN', 'HACHIN', 'CVDCOG', 'STROKCOG', 'CVDIMAG', 'CVDIMAG1', 'CVDIMAG2', 'CVDIMAG3', 'CVDIMAG4', 'PDNORMAL', 'SPEECH', 'FACEXP', 'TRESTFAC', 'TRESTRHD', 'TRESTLHD', 'TRESTRFT', 'TRESTLFT', 'TRACTRHD', 'TRACTLHD', 'RIGDNECK', 'RIGDUPRT', 'RIGDUPLF', 'RIGDLORT', 'RIGDLOLF', 'TAPSRT', 'TAPSLF', 'HANDMOVR', 'HANDMOVL', 'HANDALTR', 'HANDALTL', 'LEGRT', 'LEGLF', 'ARISING', 'POSTURE', 'GAIT', 'POSSTAB', 'BRADYKIN', 'NPIQINF', 'DELSEV', 'HALLSEV', 'AGITSEV', 'DEPDSEV', 'ELATSEV', 'APASEV', 'DISNSEV', 'IRRSEV', 'MOTSEV', 'NITESEV', 'APPSEV', 'NOGDS', 'SATIS', 'DROPACT', 'EMPTY', 'BORED', 'SPIRITS', 'AFRAID', 'HAPPY', 'HELPLESS', 'STAYHOME', 'MEMPROB', 'WONDRFUL', 'WRTHLESS', 'ENERGY', 'HOPELESS', 'BETTER', 'NACCGDS', 'BILLS', 'TAXES', 'SHOPPING', 'GAMES', 'STOVE', 'MEALPREP', 'EVENTS', 'PAYATTN', 'REMDATES', 'TRAVEL', 'NORMEXAM', 'FOCLDEF', 'GAITDIS', 'EYEMOVE', 'PARKSIGN', 'RESTTRL', 'RESTTRR', 'SLOWINGL', 'SLOWINGR', 'RIGIDL', 'RIGIDR', 'BRADY', 'PARKGAIT', 'POSTINST', 'CVDSIGNS', 'CORTDEF', 'SIVDFIND', 'CVDMOTL', 'CVDMOTR', 'CORTVISL', 'CORTVISR', 'SOMATL', 'SOMATR', 'POSTCORT', 'PSPCBS', 'EYEPSP', 'DYSPSP', 'AXIALPSP', 'GAITPSP', 'APRAXSP', 'APRAXL', 'APRAXR', 'CORTSENL', 'CORTSENR', 'ATAXL', 'ATAXR', 'ALIENLML', 'ALIENLMR', 'DYSTONL', 'DYSTONR', 'MYOCLLT', 'MYOCLRT', 'ALSFIND', 'GAITNPH', 'OTHNEUR', 'MMSECOMP', 'MMSELOC', 'MMSELAN', 'MMSEVIS', 'MMSEHEAR', 'MMSEORDA', 'MMSEORLO', 'PENTAGON', 'NACCMMSE', 'NPSYCLOC', 'NPSYLAN', 'LOGIMO', 'LOGIDAY', 'LOGIYR', 'LOGIPREV', 'LOGIMEM', 'MEMUNITS', 'MEMTIME', 'UDSBENTC', 'UDSBENTD', 'UDSBENRS', 'DIGIF', 'DIGIFLEN', 'DIGIB', 'DIGIBLEN', 'ANIMALS', 'VEG', 'TRAILA', 'TRAILARR', 'TRAILALI', 'TRAILB', 'TRAILBRR', 'TRAILBLI', 'WAIS', 'BOSTON', 'UDSVERFC', 'UDSVERFN', 'UDSVERNF', 'UDSVERLC', 'UDSVERLR', 'UDSVERLN', 'UDSVERTN', 'UDSVERTE', 'UDSVERTI', 'COGSTAT', 'MODCOMM', 'MOCACOMP', 'MOCAREAS', 'MOCALOC', 'MOCALAN', 'MOCAVIS', 'MOCAHEAR', 'MOCATOTS', 'NACCMOCA', 'MOCATRAI', 'MOCACUBE', 'MOCACLOC', 'MOCACLON', 'MOCACLOH', 'MOCANAMI', 'MOCAREGI', 'MOCADIGI', 'MOCALETT', 'MOCASER7', 'MOCAREPE', 'MOCAFLUE', 'MOCAABST', 'MOCARECN', 'MOCARECC', 'MOCARECR', 'MOCAORDT', 'MOCAORMO', 'MOCAORYR', 'MOCAORDY', 'MOCAORPL', 'MOCAORCT', 'CRAFTVRS', 'CRAFTURS', 'DIGFORCT', 'DIGFORSL', 'DIGBACCT', 'DIGBACLS', 'CRAFTDVR', 'CRAFTDRE', 'CRAFTDTI', 'CRAFTCUE', 'MINTTOTS', 'MINTTOTW', 'MINTSCNG', 'MINTSCNC', 'MINTPCNG', 'MINTPCNC', 'MOCBTOTS', 'NACCMOCB', 'REY1REC', 'REY1INT', 'REY2REC', 'REY2INT', 'REY3REC', 'REY3INT', 'REY4REC', 'REY4INT', 'REY5REC', 'REY5INT', 'REY6REC', 'REY6INT', 'OTRAILA', 'OTRLARR', 'OTRLALI', 'OTRAILB', 'OTRLBRR', 'OTRLBLI', 'REYDREC', 'REYDINT', 'REYTCOR', 'REYFPOS', 'VNTTOTW', 'VNTPCNC', 'RESPVAL', 'RESPHEAR', 'RESPDIST', 'RESPINTR', 'RESPDISN', 'RESPFATG', 'RESPEMOT', 'RESPASST', 'RESPOTH', 'NACCNE4S', 'SEX', 'ANXSEV', 'ARTHUPEX', 'ARTHLOEX', 'NACCAANX', 'NACCNREX']

In [53]:
len(nacc_orig_columns)

410

In [54]:
def get_df_orig_labels(df):
    df_orig = nacc_unique_df_type_3[nacc_unique_df_type_3['NACCID'].isin(df.ID)]
    df_orig['ID'] = df_orig['NACCID']
    df_orig['case_number'] = np.NaN
    df_orig['number'] = np.NaN

    for i, row in df_orig.iterrows():
        proc_row = df[df['ID'] == row['NACCID']]
        # print(proc_row['filename'].item())
        df_orig['case_number'][i] = proc_row['case_number'].item()
        df_orig['number'][i] = int(proc_row['case_number'].item().split('_')[1])

    df_orig['NC_lb'] = df_orig['NC']
    # df_orig['IMCI_lb'] = df_orig['IMCI']
    df_orig['MCI_lb'] = df_orig['MCI']
    df_orig['DE_lb'] = df_orig['DE']
    df_orig['AD_lb'] = df_orig['AD']
    df_orig['LBD_lb'] = df_orig['LBD']
    df_orig['VD_lb'] = df_orig['VD']
    df_orig['PRD_lb'] = df_orig['PRD']
    df_orig['FTD_lb'] = df_orig['FTD']
    df_orig['NPH_lb'] = df_orig['NPH']
    df_orig['SEF_lb'] = df_orig['SEF']
    df_orig['PSY_lb'] = df_orig['PSY']
    df_orig['TBI_lb'] = df_orig['TBI']
    df_orig['ODE_lb'] = df_orig['ODE']
    df_orig['TBI'] = df_orig['his_TBI']

    df_orig = df_orig.sort_values(['number'])
    df_orig = df_orig[nacc_orig_columns].reset_index(drop=True)
    
    return df_orig

In [55]:
cli_cases_orig_labels_neuro = get_df_orig_labels(cli_cases_neuro)

In [56]:
cli_cases_orig_labels_neuro

,filename,ID,non_null_count,case_number,MRI_CNT,NC_lb,MCI_lb,DE_lb,AD_lb,LBD_lb,...,RESPEMOT,RESPASST,RESPOTH,NACCNE4S,SEX,ANXSEV,ARTHUPEX,ARTHLOEX,NACCAANX,NACCNREX
0,mri5502.zip,NACC425502,204,CASE_1,5.0,1,0,0,0,0,...,-4,-4,-4,0,2,8,-4,-4,0,1
1,mri5263.zip,NACC784242,204,CASE_2,5.0,1,0,0,0,0,...,-4,-4,-4,0,2,8,-4,-4,1,1
2,mri3615.zip,NACC158963,199,CASE_3,6.0,0,0,1,0,0,...,-4,-4,-4,0,1,8,-4,-4,0,0
3,mri5469.zip,NACC464247,204,CASE_4,5.0,1,0,0,0,0,...,-4,-4,-4,0,2,8,-4,-4,0,1
4,mri5397.zip,NACC087355,204,CASE_5,5.0,0,1,0,0,0,...,-4,-4,-4,1,2,1,-4,-4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,mri5000.zip,NACC827850,205,CASE_96,5.0,1,0,0,0,0,...,-4,-4,-4,0,1,8,-4,-4,0,1
96,NACC630130_12840113619213417625342834029133452...,NACC630130,197,CASE_97,3.0,0,0,1,1,0,...,-4,-4,-4,1,1,2,-4,-4,0,1
97,mri5405.zip,NACC475721,205,CASE_98,5.0,1,0,0,0,0,...,-4,-4,-4,0,2,8,-4,-4,0,1
98,mri3259.zip,NACC311767,181,CASE_99,6.0,0,0,1,1,0,...,-4,-4,-4,0,2,8,-4,-4,0,1


In [57]:
# cli_cases_orig_labels_neuro['ABRUPT'].value_counts()

In [58]:
# cli_cases_neuro['cvd_ABRUPT'].value_counts()

In [59]:
cli_cases_orig_labels_neuro[cli_cases_orig_labels_neuro['case_number'] == 'CASE_10']['filename'].item()

'mri1880.zip'

In [60]:
cli_cases_orig_labels_converted_neuro = cli_cases_orig_labels_neuro.replace({-4: np.NaN, '-4':np.NaN, -4.0: np.NaN, '-4.0':np.NaN})

In [61]:
cli_cases_orig_labels_neuro.to_csv('/data_1/clinician_review/Neurologist/clinician_review_cases_neuro.csv', index=False)
cli_cases_orig_labels_converted_neuro.to_csv('/data_1/clinician_review/Neurologist/clinician_review_cases_converted_neuro.csv', index=False)

In [28]:
pd.read_csv('/data_1/clinician_review/Neurologist/clinician_review_cases_proc_fea_neuro.csv')

,filename,ID,non_null_count,MRI_CNT,NC,IMCI,MCI,DE,AD,LBD,...,bat_RESPDIST,bat_RESPINTR,bat_RESPDISN,bat_RESPFATG,bat_RESPEMOT,bat_RESPASST,bat_RESPOTH,bat_RESPOTHX,apoe_NACCNE4S,case_number
0,mri5502.zip,NACC425502,204,5.0,1,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_1
1,mri5263.zip,NACC784242,204,5.0,1,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_2
2,mri3615.zip,NACC158963,199,6.0,0,0.0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_3
3,mri5469.zip,NACC464247,204,5.0,1,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_4
4,mri5397.zip,NACC087355,204,5.0,0,NaN,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CASE_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,mri5000.zip,NACC827850,205,5.0,1,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_96
96,NACC630130_12840113619213417625342834029133452...,NACC630130,197,3.0,0,0.0,0,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CASE_97
97,mri5405.zip,NACC475721,205,5.0,1,NaN,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_98
98,mri3259.zip,NACC311767,181,6.0,0,0.0,0,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,CASE_99


In [24]:
pd.read_csv('/data_1/clinician_review/Neurologist/clinician_review_cases_converted_neuro.csv')

,filename,ID,non_null_count,case_number,MRI_CNT,NC_lb,MCI_lb,DE_lb,AD_lb,LBD_lb,...,RESPEMOT,RESPASST,RESPOTH,NACCNE4S,SEX,ANXSEV,ARTHUPEX,ARTHLOEX,NACCAANX,NACCNREX
0,mri5502.zip,NACC425502,204,CASE_1,5.0,1,0,0,0,0,...,NaN,NaN,NaN,0,2,8.0,NaN,NaN,0,1
1,mri5263.zip,NACC784242,204,CASE_2,5.0,1,0,0,0,0,...,NaN,NaN,NaN,0,2,8.0,NaN,NaN,1,1
2,mri3615.zip,NACC158963,199,CASE_3,6.0,0,0,1,0,0,...,NaN,NaN,NaN,0,1,8.0,NaN,NaN,0,0
3,mri5469.zip,NACC464247,204,CASE_4,5.0,1,0,0,0,0,...,NaN,NaN,NaN,0,2,8.0,NaN,NaN,0,1
4,mri5397.zip,NACC087355,204,CASE_5,5.0,0,1,0,0,0,...,NaN,NaN,NaN,1,2,1.0,NaN,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,mri5000.zip,NACC827850,205,CASE_96,5.0,1,0,0,0,0,...,NaN,NaN,NaN,0,1,8.0,NaN,NaN,0,1
96,NACC630130_12840113619213417625342834029133452...,NACC630130,197,CASE_97,3.0,0,0,1,1,0,...,NaN,NaN,NaN,1,1,2.0,NaN,NaN,0,1
97,mri5405.zip,NACC475721,205,CASE_98,5.0,1,0,0,0,0,...,NaN,NaN,NaN,0,2,8.0,NaN,NaN,0,1
98,mri3259.zip,NACC311767,181,CASE_99,6.0,0,0,1,1,0,...,NaN,NaN,NaN,0,2,8.0,NaN,NaN,0,1


In [62]:
cli_cases_labels_neuro = cli_cases_neuro[['filename', 'ID', 'case_number', 'NC', 'MCI', 'DE', 'AD', 'LBD', 'VD', 'PRD', 'FTD', 'NPH', 'SEF', 'PSY', 'TBI', 'ODE']]

In [63]:
cli_cases_labels_neuro.to_csv('/data_1/clinician_review/Neurologist/cli_cases_labels_neuro.csv', index=False)

### Radiologist

In [68]:
cli_cases_radio = cli_cases_neuro[cli_cases_neuro['DE'] == 1].reset_index(drop=True)
cli_cases_radio['case_number'] = 'CASE_' + (cli_cases_radio.index + 1).astype(str)

In [69]:
cli_cases_orig_labels_radio = get_df_orig_labels(cli_cases_radio)
cli_cases_orig_labels_converted_radio = cli_cases_orig_labels_radio.replace({-4: np.NaN, '-4':np.NaN, -4.0: np.NaN, '-4.0':np.NaN})

In [70]:
cli_cases_radio.to_csv('/data_1/clinician_review/Radiologist/clinician_review_cases_proc_fea_radio.csv', index=False)
cli_cases_orig_labels_radio.to_csv('/data_1/clinician_review/Radiologist/clinician_review_cases_radio.csv', index=False)
cli_cases_orig_labels_converted_radio.to_csv('/data_1/clinician_review/Radiologist/clinician_review_cases_converted_radio.csv', index=False)

In [71]:
cli_cases_neuro[cli_cases_neuro['case_number'] == 'CASE_61'][['NC', 'IMCI', 'MCI', 'DE', 'AD', 'LBD', 'VD', 'PRD', 'FTD', 'NPH', 'SEF', 'PSY', 'TBI', 'ODE']]

,NC,IMCI,MCI,DE,AD,LBD,VD,PRD,FTD,NPH,SEF,PSY,TBI,ODE
60,0,0,0,1,1,0,0,0,0,0,0,0,1,0


In [72]:
cli_cases_orig_labels_converted_radio

,filename,ID,non_null_count,case_number,MRI_CNT,NC_lb,IMCI_lb,MCI_lb,DE_lb,AD_lb,...,RESPEMOT,RESPASST,RESPOTH,NACCNE4S,SEX,ANXSEV,ARTHUPEX,ARTHLOEX,NACCAANX,NACCNREX
0,mri6873.zip,NACC433881,198,CASE_1,4.0,0,0,0,1,1,...,NaN,NaN,NaN,1,2,1.0,NaN,NaN,0,1
1,NACC670524_12840113696323972500152388820101028...,NACC670524,183,CASE_2,3.0,0,0,0,1,1,...,NaN,NaN,NaN,9,2,2.0,NaN,NaN,0,1
2,NACC698052_21684011141514153893144971165483.zip,NACC698052,191,CASE_3,2.0,0,0,0,1,1,...,NaN,NaN,NaN,9,2,3.0,NaN,NaN,0,1
3,NACC877796_12840113745101000100200039925398826...,NACC877796,201,CASE_4,4.0,0,0,0,1,1,...,NaN,NaN,NaN,0,1,8.0,NaN,NaN,0,1
4,mri3617.zip,NACC884172,170,CASE_5,6.0,0,0,0,1,0,...,NaN,NaN,NaN,0,2,3.0,NaN,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,mri1880.zip,NACC590625,198,CASE_66,5.0,0,0,0,1,0,...,NaN,NaN,NaN,0,2,1.0,NaN,NaN,0,0
66,mri3582.zip,NACC769021,196,CASE_67,6.0,0,0,0,1,0,...,NaN,NaN,NaN,0,1,8.0,NaN,NaN,0,1
67,mri3573.zip,NACC318822,191,CASE_68,6.0,0,0,0,1,0,...,NaN,NaN,NaN,9,2,2.0,NaN,NaN,1,1
68,NACC209327_12840113745101000100200040113592627...,NACC209327,200,CASE_69,4.0,0,0,0,1,1,...,NaN,NaN,NaN,0,2,8.0,NaN,NaN,0,0


In [73]:
set(cli_cases_orig_labels_neuro[cli_cases_orig_labels_converted_neuro['DE_lb'] == 1]['filename']) - set(cli_cases_orig_labels_converted_radio['filename'])

set()

In [78]:
# for label in ['TBI', 'NPH', 'PRD', 'AD', 'LBD', 'VD', 'FTD', 'SEF', 'PSY', 'ODE']:
#     print(cli_cases_neuro[label].value_counts())

In [75]:
len(set(cli_cases_radio['ID']))

70

### Save cases

#### 3d mris

In [40]:
# mris = list(cli_cases.filename)
# mris = [mri.split('.')[0] + 'ni' for mri in mris]
# cli_mris_3d = {mri: {'t1': [], 't2': [], 'dti': [], 'dwi': [], 'swi': [], 'flair': [], 'unknown': []} for mri in mris}
# for mri in mris:
#     for mag, seq_dict in mri_3d.items():
#         for seq, mri_dict in seq_dict.items():
#             for mri_name, mri_list in mri_dict.items():
#                 if mri_name == mri:
#                     cli_mris_3d[mri][seq] += mri_list

# for mri in mris:
#     if (len(cli_mris_3d[mri]['t1']) != 0) | (len(cli_mris_3d[mri]['t2']) != 0) | (len(cli_mris_3d[mri]['dti']) != 0) | (len(cli_mris_3d[mri]['dwi']) != 0) | (len(cli_mris_3d[mri]['flair']) != 0) | (len(cli_mris_3d[mri]['swi']) != 0) | (len(cli_mris_3d[mri]['unknown']) != 0):
#         continue
#     for mag, seq_dict in mri_2d.items():
#         for seq, mri_dict in seq_dict.items():
#             for mri_name, mri_list in mri_dict.items():
#                 if mri_name == mri:
#                     cli_mris_3d[mri][seq] += mri_list

# for mri in mris:
#     if (len(cli_mris_3d[mri]['t1']) != 0) | (len(cli_mris_3d[mri]['t2']) != 0) | (len(cli_mris_3d[mri]['dti']) != 0) | (len(cli_mris_3d[mri]['dwi']) != 0) | (len(cli_mris_3d[mri]['flair']) != 0) | (len(cli_mris_3d[mri]['swi']) != 0) | (len(cli_mris_3d[mri]['unknown']) != 0):
#         continue
#     for mag, seq_dict in mri_unk.items():
#         for seq, mri_dict in seq_dict.items():
#             for mri_name, mri_list in mri_dict.items():
#                 if mri_name == mri:
#                     cli_mris_3d[mri][seq] += mri_list

In [ ]:
# cli_mris_3d

In [51]:
# import shutil
# dest = '/home/skowshik/ADRD_repo/pipeline_v1_temp/clinician_review/clinician_review_case_mris_3d/'
# for i, (mri_name, mris) in enumerate(cli_mris_3d.items()):
#     for seq, mri_list in mris.items():
#         case_no = cli_cases[cli_cases['filename'] == mri_name[:-2] + '.zip']['case_number'].item()
#         folder_path = dest + case_no + '/'
#         # print(folder_path)
#         if not os.path.exists(folder_path):
#             os.makedirs(folder_path)
#         # print(mri_list)
#         for j, mri in enumerate(mri_list):
#             # print(mri)
#             dest_path = folder_path + seq + '_' + str(j+1)
#             shutil.copy(mri['mri'], dest_path + '.nii')
#             shutil.copy(mri['json'], dest_path + '.json')

#### 3d + 2d MRIs

In [14]:
cli_cases_orig_labels_converted_neuro = pd.read_csv('/data_1/clinician_review/Neurologist/clinician_review_cases_converted_neuro.csv')
cli_cases_orig_labels_converted_radio = pd.read_csv('/data_1/clinician_review/Radiologist/clinician_review_cases_converted_radio.csv')

In [15]:
len(set(set(cli_cases_orig_labels_converted_neuro[cli_cases_orig_labels_converted_neuro['MCI_lb'] == 0]['ID']) - (set(cli_cases_orig_labels_converted_radio['ID']))))

15

In [21]:
len(set(cli_cases_orig_labels_converted_neuro['ID']) - set(nacc_unique_df_type_3['ID']))

0

In [14]:
pd.read_csv('/data_1/clinician_review/Neurologist/clinician_review_cases_proc_fea_neuro.csv')['his_TBI'].value_counts()

0.0    5
2.0    1
Name: his_TBI, dtype: int64

In [15]:
cli_cases_orig_labels_converted_neuro['TBI'].value_counts()

0.0    5
2.0    1
Name: TBI, dtype: int64

In [6]:
radio = {}
for label in ['NC_lb', 'MCI_lb', 'DE_lb', 'AD_lb', 'LBD_lb', 'VD_lb', 'PRD_lb', 'FTD_lb', 'NPH_lb', 'SEF_lb', 'PSY_lb', 'TBI_lb', 'ODE_lb']:
    radio[label] = dict(cli_cases_orig_labels_converted_radio[label].value_counts())
radio

{'NC_lb': {0: 70},
 'MCI_lb': {0: 70},
 'DE_lb': {1: 70},
 'AD_lb': {1: 41, 0: 29},
 'LBD_lb': {0: 61, 1: 9},
 'VD_lb': {0: 59, 1: 11},
 'PRD_lb': {0: 63, 1: 7},
 'FTD_lb': {0: 59, 1: 11},
 'NPH_lb': {0: 63, 1: 7},
 'SEF_lb': {0: 62, 1: 8},
 'PSY_lb': {0: 55, 1: 15},
 'TBI_lb': {0: 63, 1: 7},
 'ODE_lb': {0: 61, 1: 9}}

In [7]:
neuro = {}
for label in ['NC_lb', 'MCI_lb', 'DE_lb', 'AD_lb', 'LBD_lb', 'VD_lb', 'PRD_lb', 'FTD_lb', 'NPH_lb', 'SEF_lb', 'PSY_lb', 'TBI_lb', 'ODE_lb']:
    neuro[label] = dict(cli_cases_orig_labels_converted_neuro[label].value_counts())
neuro

{'NC_lb': {0: 85, 1: 15},
 'MCI_lb': {0: 85, 1: 15},
 'DE_lb': {1: 70, 0: 30},
 'AD_lb': {0: 59, 1: 41},
 'LBD_lb': {0: 91, 1: 9},
 'VD_lb': {0: 89, 1: 11},
 'PRD_lb': {0: 93, 1: 7},
 'FTD_lb': {0: 89, 1: 11},
 'NPH_lb': {0: 93, 1: 7},
 'SEF_lb': {0: 92, 1: 8},
 'PSY_lb': {0: 85, 1: 15},
 'TBI_lb': {0: 93, 1: 7},
 'ODE_lb': {0: 91, 1: 9}}

In [39]:
nc_mci = set(cli_cases_orig_labels_converted_neuro['filename']) - set(cli_cases_orig_labels_converted_radio['filename'])

In [40]:
cli_cases_orig_labels_converted_neuro

,filename,ID,non_null_count,case_number,MRI_CNT,NC_lb,MCI_lb,DE_lb,AD_lb,LBD_lb,...,RESPEMOT,RESPASST,RESPOTH,NACCNE4S,SEX,ANXSEV,ARTHUPEX,ARTHLOEX,NACCAANX,NACCNREX
0,mri5502.zip,NACC425502,204,CASE_1,5.0,1,0,0,0,0,...,NaN,NaN,NaN,0,2,8.0,NaN,NaN,0,1
1,mri5263.zip,NACC784242,204,CASE_2,5.0,1,0,0,0,0,...,NaN,NaN,NaN,0,2,8.0,NaN,NaN,1,1
2,mri3615.zip,NACC158963,199,CASE_3,6.0,0,0,1,0,0,...,NaN,NaN,NaN,0,1,8.0,NaN,NaN,0,0
3,mri5469.zip,NACC464247,204,CASE_4,5.0,1,0,0,0,0,...,NaN,NaN,NaN,0,2,8.0,NaN,NaN,0,1
4,mri5397.zip,NACC087355,204,CASE_5,5.0,0,1,0,0,0,...,NaN,NaN,NaN,1,2,1.0,NaN,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,mri5000.zip,NACC827850,205,CASE_96,5.0,1,0,0,0,0,...,NaN,NaN,NaN,0,1,8.0,NaN,NaN,0,1
96,NACC630130_12840113619213417625342834029133452...,NACC630130,197,CASE_97,3.0,0,0,1,1,0,...,NaN,NaN,NaN,1,1,2.0,NaN,NaN,0,1
97,mri5405.zip,NACC475721,205,CASE_98,5.0,1,0,0,0,0,...,NaN,NaN,NaN,0,2,8.0,NaN,NaN,0,1
98,mri3259.zip,NACC311767,181,CASE_99,6.0,0,0,1,1,0,...,NaN,NaN,NaN,0,2,8.0,NaN,NaN,0,1


In [41]:
# prev = pd.read_csv('/data_1/clinician_review/temp/clinician_review_cases_converted_neuro.csv')
# other = list(set(cli_cases_orig_labels_converted_neuro['ID']) - set(prev['ID']))
cases = []
for i, row in cli_cases_orig_labels_converted_neuro.iterrows():
    if row['filename'] in nc_mci:
        cases.append(row['case_number'])

In [ ]:
cases

In [43]:
check = cli_cases_orig_labels_converted_neuro[cli_cases_orig_labels_converted_neuro['filename'].isin(nc_mci)]

In [44]:
len(set(check['filename']) - nc_mci)

0

In [45]:
def get_cli_mris_dict(cases):
    mris = list(cases.filename)
    mris = [mri.split('.')[0] + 'ni' for mri in mris]
    cli_mris_all = {mri: {'t1': [], 't2': [], 'dti': [], 'dwi': [], 'swi': [], 'flair': [], 'unknown': []} for mri in mris}
    for mri in mris:
        for mag, seq_dict in mri_3d.items():
            for seq, mri_dict in seq_dict.items():
                for mri_name, mri_list in mri_dict.items():
                    if mri_name == mri:
                        cli_mris_all[mri][seq] += mri_list

    for mri in mris:
        for mag, seq_dict in mri_2d.items():
            for seq, mri_dict in seq_dict.items():
                for mri_name, mri_list in mri_dict.items():
                    if mri_name == mri:
                        cli_mris_all[mri][seq] += mri_list

    for mri in mris:
        for mag, seq_dict in mri_unk.items():
            for seq, mri_dict in seq_dict.items():
                for mri_name, mri_list in mri_dict.items():
                    if mri_name == mri:
                        cli_mris_all[mri][seq] += mri_list
    return cli_mris_all

In [46]:
import shutil
def save_mris(cli_mris, dest, cases):
    # dest = '/data_1/clinician_review/clinician_review_mris/'
    for i, (mri_name, mris) in enumerate(cli_mris.items()):
        case_no = cases[cases['filename'] == mri_name[:-2] + '.zip']['case_number'].item()
        k = 1
        for seq, mri_list in mris.items():
            folder_path = dest + case_no + '/'
            # print(folder_path)
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            # else:
            #     continue
            # print(folder_path)
            for j, mri in enumerate(mri_list):
                with open(mri['json'], 'r') as fi:
                    data = json.load(fi)
                    series = data['SeriesDescription']
                    if 'MRAcquisitionType' in data:
                        ack = data["MRAcquisitionType"]
                        dest_path = folder_path + case_no + '_' + series + '_' + ack + '_' + str(k)
                    else:
                        dest_path = folder_path + case_no + '_' + series + '_' + str(k)
                    # print(dest_path)
                    shutil.copy(mri['mri'], dest_path + '.nii')
                    # shutil.copy(mri['json'], dest_path + '.json')
                    k += 1

In [47]:
cli_mris_neuro = get_cli_mris_dict(cli_cases_orig_labels_converted_neuro)

In [ ]:
cli_mris_neuro

In [36]:
dest_neuro = '/data_1/clinician_review/Neurologist/clinician_review_neurologist_mris/'
save_mris(cli_mris_neuro, dest_neuro, cli_cases_orig_labels_converted_neuro)

In [39]:
cli_mris_radio = get_cli_mris_dict(cli_cases_orig_labels_converted_radio)

In [ ]:
cli_mris_radio

In [41]:
dest_radio = '/data_1/clinician_review/Radiologist/clinician_review_radiologist_mris/'
save_mris(cli_mris_radio, dest_radio, cli_cases_orig_labels_converted_radio)

In [73]:
cli_cases_orig_labels_converted_radio

,filename,ID,non_null_count,case_number,MRI_CNT,NC_lb,IMCI_lb,MCI_lb,DE_lb,AD_lb,...,RESPEMOT,RESPASST,RESPOTH,NACCNE4S,SEX,ANXSEV,ARTHUPEX,ARTHLOEX,NACCAANX,NACCNREX
0,mri6873.zip,NACC433881,198,CASE_1,4.0,0,0,0,1,1,...,NaN,NaN,NaN,1,2,1.0,NaN,NaN,0,1
1,NACC670524_12840113696323972500152388820101028...,NACC670524,183,CASE_2,3.0,0,0,0,1,1,...,NaN,NaN,NaN,9,2,2.0,NaN,NaN,0,1
2,NACC698052_21684011141514153893144971165483.zip,NACC698052,191,CASE_3,2.0,0,0,0,1,1,...,NaN,NaN,NaN,9,2,3.0,NaN,NaN,0,1
3,NACC877796_12840113745101000100200039925398826...,NACC877796,201,CASE_4,4.0,0,0,0,1,1,...,NaN,NaN,NaN,0,1,8.0,NaN,NaN,0,1
4,mri3617.zip,NACC884172,170,CASE_5,6.0,0,0,0,1,0,...,NaN,NaN,NaN,0,2,3.0,NaN,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,mri1880.zip,NACC590625,198,CASE_66,5.0,0,0,0,1,0,...,NaN,NaN,NaN,0,2,1.0,NaN,NaN,0,0
66,mri3582.zip,NACC769021,196,CASE_67,6.0,0,0,0,1,0,...,NaN,NaN,NaN,0,1,8.0,NaN,NaN,0,1
67,mri3573.zip,NACC318822,191,CASE_68,6.0,0,0,0,1,0,...,NaN,NaN,NaN,9,2,2.0,NaN,NaN,1,1
68,NACC209327_12840113745101000100200040113592627...,NACC209327,200,CASE_69,4.0,0,0,0,1,1,...,NaN,NaN,NaN,0,2,8.0,NaN,NaN,0,0


In [57]:
# mris = list(cli_cases.filename)
# mris = [mri.split('.')[0] + 'ni' for mri in mris]
# cli_mris_all = {mri: {'t1': [], 't2': [], 'dti': [], 'dwi': [], 'swi': [], 'flair': [], 'unknown': []} for mri in mris}
# for mri in mris:
#     for mag, seq_dict in mri_3d.items():
#         for seq, mri_dict in seq_dict.items():
#             for mri_name, mri_list in mri_dict.items():
#                 if mri_name == mri:
#                     cli_mris_all[mri][seq] += mri_list

# for mri in mris:
#     for mag, seq_dict in mri_2d.items():
#         for seq, mri_dict in seq_dict.items():
#             for mri_name, mri_list in mri_dict.items():
#                 if mri_name == mri:
#                     cli_mris_all[mri][seq] += mri_list

# for mri in mris:
#     for mag, seq_dict in mri_unk.items():
#         for seq, mri_dict in seq_dict.items():
#             for mri_name, mri_list in mri_dict.items():
#                 if mri_name == mri:
#                     cli_mris_all[mri][seq] += mri_list

In [59]:
# import shutil
# dest = '/data_1/clinician_review/clinician_review_mris/'
# for i, (mri_name, mris) in enumerate(cli_mris_all.items()):
#     case_no = cli_cases[cli_cases['filename'] == mri_name[:-2] + '.zip']['case_number'].item()
#     k = 1
#     for seq, mri_list in mris.items():
#         folder_path = dest + case_no + '/'
#         # print(folder_path)
#         if not os.path.exists(folder_path):
#             os.makedirs(folder_path)
#         # print(mri_list)
#         for j, mri in enumerate(mri_list):
#             with open(mri['json'], 'r') as fi:
#                 data = json.load(fi)
#                 series = data['SeriesDescription']
#                 if 'MRAcquisitionType' in data:
#                     ack = data["MRAcquisitionType"]
#                     dest_path = folder_path + case_no + '_' + series + '_' + ack + '_' + str(k)
#                 else:
#                     dest_path = folder_path + case_no + '_' + series + '_' + str(k)
#                 # print(dest_path)
#                 shutil.copy(mri['mri'], dest_path + '.nii')
#                 # shutil.copy(mri['json'], dest_path + '.json')
#                 k += 1
#     #     # break
#     # if i == 2:
#     #     break